# Train Model
## i. Overview
The Model Trainer is used to generate and train a ML model for our use case. The data is assumed to already be preprocessed and do not need much additional manipulation.

## ii. Special Notes
1. The file is always expected to be loaded as a dense matrix. For better processing speed, we can convert the dense matrix into a sparse matrix.
2. We need to figure out how to improve processing speed while keeping up performance.

## iii. Methodology
### Selected Algorithm

### Additional





In [3]:
#import libraries
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import imblearn as imbl
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import RUSBoostClassifier

trainDataPath = 'data/trainData_2018-2019'
trainDataPathBinaryTarget = 'data/trainDataBinaryTarget_2018-2019'

selectedFeatures = [
    'Airline',
    'Origin',
    'Dest',
    'CRSDepTime', 
    'Distance', 
    'Year', 
    'Quarter', 
    'Month', 
    'DayofMonth', 
    'DayOfWeek', 
    'DepTimeBlk', 
    'ArrTimeBlk', 
    'DistanceGroup'
    ]

In [4]:
#load data and set up for training
trainData = pd.read_parquet(trainDataPath, engine='fastparquet')
print(f'Train data shape: {trainData.shape}')

trainDataBinaryTarget = pd.read_parquet(trainDataPathBinaryTarget, engine='fastparquet')
print(f'Binary target train data shape: {trainDataBinaryTarget.shape}')

Train data shape: (5697559, 15)
Binary target train data shape: (13503883, 15)


In [5]:
#define any relevant functions

# SaveModel
#
# function that can be used to save the model that we just built
# after fitting the model, save it to filename
import pickle
def SaveModel(model, filename):
    pickle.dump(model, open(filename, 'wb'))

#CreateModelUsingCrossVal
#
#Creates a model given the input of
#x features, y target, and which type of
#model to run. Then uses cross validation
#to run that model and prints out the scores.
def CreateModelUsingCrossVal(trainx, trainy, mode):
    if (mode == 'logistic_regression' or mode == 'lr'):
        model = skl.linear_model.LogisticRegression(max_iter=1000)
    elif (mode == 'naive_bayes' or mode == 'nb'):
        model = skl.naive_bayes.GaussianNB()
    elif (mode == 'random_forest' or mode == 'rf'):
        model = skl.ensemble.RandomForestClassifier(max_depth=2)
    elif (mode == 'balanced_random_forest' or mode == 'brf'):
        model = BalancedRandomForestClassifier(max_depth=2)
    elif (mode == 'adaboost' or mode == 'ab'):
        model = RUSBoostClassifier()
    
    model.fit(trainx,trainy)
    SaveModel(model, "models/multiclass_model_"+mode+".pkl")

    desiredScores=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

    results=cross_validate(model, trainx, trainy, cv=5, scoring=desiredScores, return_train_score=False)

    print(f"Model Type: {mode}")
    print("Scores from cross-validation are:")
    print(f'Accuracy: {results["test_accuracy"].mean()}')
    print(f'Precision: {results["test_precision_macro"].mean()}')
    print(f'Recall: {results["test_recall_macro"].mean()}')
    print(f'F-1 Score: {results["test_f1_macro"].mean()}')

# Function to be used for models
def BinaryTargetCreateModelUsingCrossVal(trainx, trainy, mode):
    if (mode == 'logistic_regression' or mode == 'lr'):
        model = skl.linear_model.LogisticRegression(max_iter=1000)
    elif (mode == 'naive_bayes' or mode == 'nb'):
        model = skl.naive_bayes.GaussianNB()
    elif (mode == 'random_forest' or mode == 'rf'):
        model = skl.ensemble.RandomForestClassifier(max_depth=2)
    elif (mode == 'balanced_random_forest' or mode == 'brf'):
        model = BalancedRandomForestClassifier(max_depth=2)
    elif (mode == 'adaboost' or mode == 'ab'):
        model = RUSBoostClassifier()
    
    fit=model.fit(trainx,trainy)
    model.fit(trainx,trainy)
    SaveModel(model, "models/binary_model_"+mode+".pkl")

    desiredScores=['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

    results=cross_validate(model, trainx, trainy, cv=5, scoring=desiredScores, return_train_score=False)

    print(f"Model Type: {mode}")
    print("Scores from cross-validation are:")
    print(f'Accuracy: {results["test_accuracy"].mean()}')
    print(f'Precision: {results["test_precision"].mean()}')
    print(f'Recall: {results["test_recall"].mean()}')
    print(f'F-1 Score: {results["test_f1"].mean()}')
    print(f'AUC: {results["test_roc_auc"].mean()}')

In [7]:
#build model 1
xFeatures = trainData[selectedFeatures]
yTarget = np.ravel(trainData['BinArrDelayMinutes'])

print('MultiClass Evaluation')
CreateModelUsingCrossVal(xFeatures, yTarget, "naive_bayes")
print()

xFeatures = trainDataBinaryTarget[selectedFeatures]
yTarget = np.ravel(trainDataBinaryTarget['BinaryArrDelayMinutes'])
print('Binary Class Evaluation')
BinaryTargetCreateModelUsingCrossVal(xFeatures, yTarget, "naive_bayes")

MultiClass Evaluation
Model Type: naive_bayes
Scores from cross-validation are:
Accuracy: 0.19063655341326452
Precision: 0.17133174718571303
Recall: 0.18882033687287042
F-1 Score: 0.1395492648544096

Binary Class Evaluation
Model Type: naive_bayes
Scores from cross-validation are:
Accuracy: 0.6074798637203889
Precision: 0.42012410142643086
Recall: 0.30514585656887355
F-1 Score: 0.35269973709281305
AUC: 0.5773607483268706


In [9]:
#build model 2
smallerSampleDataframe = trainData.sample(n=100000)
xFeatures = smallerSampleDataframe[selectedFeatures]
yTarget = np.ravel(smallerSampleDataframe['BinArrDelayMinutes'])

print('MultiClass Evaluation')
CreateModelUsingCrossVal(xFeatures, yTarget, "logistic_regression")
print()

smallerSampleDataframe = trainDataBinaryTarget.sample(n=100000)
xFeatures = smallerSampleDataframe[selectedFeatures]
yTarget = np.ravel(smallerSampleDataframe['BinaryArrDelayMinutes'])
print('Binary Class Evaluation')
CreateModelUsingCrossVal(xFeatures, yTarget, "logistic_regression")


MultiClass Evaluation
Model Type: logistic_regression
Scores from cross-validation are:
Accuracy: 0.24674000000000001
Precision: 0.22455608099345886
Recall: 0.2400259193942041
F-1 Score: 0.19156020407977556

Binary Class Evaluation


c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model Type: logistic_regression
Scores from cross-validation are:
Accuracy: 0.65071
Precision: 0.3753597605633857
Recall: 0.49999894627781105
F-1 Score: 0.3942274096637292


c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
#build model 3
smallerSampleDataframe = trainData.sample(n=100000)
xFeatures = smallerSampleDataframe[selectedFeatures]
yTarget = np.ravel(smallerSampleDataframe['BinArrDelayMinutes'])

print('MultiClass Evaluation')
CreateModelUsingCrossVal(xFeatures, yTarget, "random_forest")
print()

smallerSampleDataframe = trainDataBinaryTarget.sample(n=100000)
xFeatures = smallerSampleDataframe[selectedFeatures]
yTarget = np.ravel(smallerSampleDataframe['BinaryArrDelayMinutes'])
print('Binary Class Evaluation')
CreateModelUsingCrossVal(xFeatures, yTarget, "logistic_regression")

MultiClass Evaluation


c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

Model Type: random_forest
Scores from cross-validation are:
Accuracy: 0.24645999999999998
Precision: 0.1461519602032994
Recall: 0.2412057481158175
F-1 Score: 0.17424708788716242

Binary Class Evaluation


c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\chrolph\Projects\CSS 581\CSS581-FlightDelayML\.venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

Model Type: logistic_regression
Scores from cross-validation are:
Accuracy: 0.6488
Precision: 0.32440324391219566
Recall: 0.49999229346485824
F-1 Score: 0.3934983015721468
